Thermal Convection
======

This example solves 2D dimensionless isoviscous thermal convection with a Rayleigh number of $10^4$, see Blankenbach *et al.* 1989 for details.

**Keywords:** material parameters, Stokes system, advective diffusive systems

**References**

B. Blankenbach, F. Busse, U. Christensen, L. Cserepes, D. Gunkel, U. Hansen, H. Harder, G. Jarvis, M. Koch, G. Marquart, D. Moore, P. Olson, H. Schmeling and T. Schnaubelt. A benchmark comparison for mantle convection codes. Geophysical Journal International, 98, 1, 23–38, 1989
http://onlinelibrary.wiley.com/doi/10.1111/j.1365-246X.1989.tb05511.x/abstract

Please run the cell below if you do not see the interface under the Setup parameters.

In [1]:
import underworld as uw
from underworld import function as fn
import underworld.visualisation as vis
import math
import ipywidgets as widgets
from IPython.display import Javascript, display, clear_output

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+3)'))

button = widgets.Button(description="Refresh Setup")
button.on_click(run_all)
display(button)

Button(description='Refresh Setup', style=ButtonStyle())

In [2]:

style = {'description_width': '100px'}

plume = widgets.FloatSlider(0.2, min = 0, max =0.5, step = 0.1, description ='plume_thickness', style = style)
slab = widgets.FloatSlider(0.2, min = 0, max = 0.5, step = 0.1, description ='slab_thickness', style = style)

tempMin = widgets.IntText(
                            value=0,
                            description='tempMin',
                            disabled=False
                            )



tempMax = widgets.IntText(
                            value=2,
                            description='tempMax',
                            disabled=False
                            )


def start_sim(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

start_button = widgets.Button(description="Start Simulation")
start_button.on_click(start_sim)



def initialise(plume, slab, tempMin = tempMin, tempMax = tempMax):
    global velocityField
    global pressureField
    global temperatureField
    global temperatureDotField
    global velBC
    global tempBC
    global mesh
    global conditions
    global tempDif
    # Set python variables for model parameters.
    boxHeight = 1.0
    boxLength = 1.0
    res = 16


    # build a rectangular domain, discretised with a Q1/dQ0 elementType
    mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"), 
                                     elementRes  = (res, res), 
                                     minCoord    = (0., 0.), 
                                     maxCoord    = (boxLength, boxHeight))

    # create mesh variables.  Note the pressure field uses the sub-mesh (dQ0)
    
    
    velocityField       = mesh.add_variable(         nodeDofCount=2 )
    pressureField       = mesh.subMesh.add_variable( nodeDofCount=1 )
    temperatureField    = mesh.add_variable(         nodeDofCount=1 )
    temperatureDotField = mesh.add_variable(         nodeDofCount=1 ) # required for AdvectionDiffusion (SUPG) method

    # Initialise values
    velocityField.data[:]       = [0.,0.]
    pressureField.data[:]       = 0.
    temperatureDotField.data[:] = 0.

    # pertStrength = 0.2
    # deltaTemp = tempMax - tempMin
    # for index, coord in enumerate(mesh.data):
    #     pertCoeff = math.cos( math.pi*0.5 * coord[0] )
    #     temperatureField.data[index] = tempMin + deltaTemp*(boxHeight - coord[1]) + pertStrength * pertCoeff
    #     temperatureField.data[index] = max(tempMin, min(tempMax, temperatureField.data[index]))


    lightIndex = tempMin 
    denseIndex = tempMax
    mediumIndex = int((lightIndex + denseIndex) / 2)
    
    tempDif = float(denseIndex - lightIndex)
    
    # material perturbation from van Keken et al. 1997
    wavelength = 2.0*boxLength
    amplitude  = 0.04
    offset_low = plume
    offset_high= 1-slab
    k = 2 * math.pi / wavelength

    # Create function to return particle's coordinate
    coord = fn.coord()

    # Define the material perturbation, a function of the x coordinate (accessed by `coord[0]`).
    perturbationFn_low = offset_low   + amplitude*fn.math.cos(k*coord[0] + math.pi/4)
    perturbationFn_high = offset_high + amplitude*fn.math.cos(k*coord[0] - math.pi/4)

    # Setup the conditions list. 
    # If z is less than the perturbation, set to lightIndex.
    conditions = [ (  perturbationFn_high < coord[1],  lightIndex),
                   (  perturbationFn_high > coord[1] > perturbationFn_low, mediumIndex ),
                   (  True, denseIndex ) ]

    # The swarm is passed as an argument to the evaluation, providing evaluation on each particle.
    # Results are written to the materialIndex swarm variable.
    temperatureField.data[:] = fn.branching.conditional( conditions ).evaluate(mesh)

    bottomWall = mesh.specialSets["MinJ_VertexSet"]
    topWall    = mesh.specialSets["MaxJ_VertexSet"]
    leftWall   = mesh.specialSets["MinI_VertexSet"]
    rightWall  = mesh.specialSets["MaxI_VertexSet"]

    # for index in bottomWall:
    #     temperatureField.data[index] = tempMax
    # for index in topWall:
    #     temperatureField.data[index] = tempMin

    # Construct sets for I (vertical) and J (horizontal) walls.
    iWalls = leftWall   + rightWall
    jWalls = bottomWall + topWall

    # 2D velocity vector can have two Dirichlet conditions on each vertex, 
    # v_x is fixed on the iWalls (vertical), v_y is fixed on the jWalls (horizontal)
    velBC  = uw.conditions.DirichletCondition( variable        = velocityField, 
                                               indexSetsPerDof = (iWalls, jWalls) )

    # Temperature is held constant on the jWalls
    tempBC = uw.conditions.DirichletCondition( variable        = temperatureField, 
                                               indexSetsPerDof = (jWalls,) )
    display(start_button)
    figtemp = vis.Figure( figsize=(800,400), title ='Initial Conditions, temperature plot' )
    figtemp.append( vis.objects.Surface(mesh, temperatureField, colours="blue white red") )
    figtemp.append( vis.objects.Mesh(mesh) )
    figtemp.show()
    
initial_button = widgets.interact_manual(initialise,
                 plume = plume,
                 slab = slab,
                 tempMin = tempMin,
                 tempMax = tempMax,
                )
initial_button.widget.children[4].description = 'Initialise the setup'
display(initial_button)



interactive(children=(FloatSlider(value=0.2, description='plume_thickness', max=0.5, style=SliderStyle(descrip…

<function __main__.initialise(plume, slab, tempMin=IntText(value=0, description='tempMin'), tempMax=IntText(value=2, description='tempMax'))>

In [4]:
# Setup python variabel and underworld.`function`
Ra = widgets.FloatLogSlider(1.0e4, base = 10, min = 1, max = 10, step = 1, description ='Ra')


def restart_sim(ev):
#         clear_output()
    temperatureField.data[:] = fn.branching.conditional( conditions ).evaluate(mesh)
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(),\
                                                                IPython.notebook.get_selected_index()+1)'))

restart_button = widgets.Button(description="Restart Simulation")
restart_button.on_click(restart_sim)
display(restart_button)


def simulation(Ra):
    time = 0.
    step = 0
    steps_end = 20
    # Set viscosity to be a constant.
    viscosity = 1.

    # Rayleigh number.
    # Construct our density `function`
    densityFn = Ra * temperatureField
    # '*' is overloaded. A multiplication of temperatureField (type MeshVariable)
    # and a python float creates a `function`. See user_guide `Function` for more details

    # Define our vertical unit vector using a python tuple (this will be converted to a function next).
    z_hat = ( 0.0, 1.0 )

    # a buoyancy force vector using the densityFn and the vertical unit vector. 
    buoyancyFn = densityFn * z_hat

    stokes = uw.systems.Stokes( velocityField = velocityField, 
                                pressureField = pressureField,
                                conditions    = velBC,
                                fn_viscosity  = viscosity, 
                                fn_bodyforce  = buoyancyFn )

    # get the default stokes equation solver
    solver = uw.systems.Solver( stokes )

    advDiff = uw.systems.AdvectionDiffusion( phiField       = temperatureField, 
                                             phiDotField    = temperatureDotField, 
                                             velocityField  = velocityField, 
                                             fn_diffusivity = 1.0, 
                                             conditions     = tempBC )

    # define an update function
    def update():
        # Retrieve the maximum possible timestep for the advection-diffusion system.
        dt = advDiff.get_max_dt()
        # Advect using this timestep size.
        advDiff.integrate(dt)
        return time+dt, step+1

    # init these guys
    # perform timestepping
    while step < steps_end:
        # Solve for the velocity field given the current temperature field.
        solver.solve()
        time, step = update()
    solver = uw.systems.Solver( stokes )
    # plot figure
    figtemp = vis.Figure( figsize=(800,400) , title = 'Convection simulation, temperature plot')
    figtemp.append( vis.objects.Surface(mesh, temperatureField, colours="blue white red") )
    figtemp.append( vis.objects.VectorArrows(mesh, velocityField/Ra*50.0/(tempDif), arrowHead=0.2, scaling=0.1) )
    figtemp.show()
    
    
run_button = widgets.interact_manual(
                        simulation,
                        Ra = Ra,
                        )
run_button.widget.children[1].description = 'Run 20 timesteps'
display(run_button)

            


<IPython.core.display.Javascript object>

Button(description='Restart Simulation', style=ButtonStyle())

interactive(children=(FloatLogSlider(value=10000.0, description='Ra', max=10.0, min=1.0, step=1.0), Button(des…

<function __main__.simulation(Ra)>